In [1]:
from qiskit import QuantumCircuit, transpile, execute, IBMQ
from states import Key
from global_parameters import GlobalParameters
from encryption_circuit import encrypt
from decryption_circuit import create_decryption_circuit
from deletion_circuit import delete
from attack_circuit import breidbart_measurement
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeMontreal
from qiskit.circuit import qpy_serialization
from utils import export_counts, random_bit_string
import os
import shutil
import json
from datetime import datetime
import experiment

In [2]:
# Experiment parameters
experiment_id = "exp2"
execution_datetime = datetime.now()
system_string = "AerSimulator"

# Qiskit parameters
provider = IBMQ.load_account()

# Ideal simulator
backend = AerSimulator()

# Simulator with noise model
# noise_model = FakeMontreal()
# backend = AerSimulator.from_backend(noise_model)

# Real backend
# backend = provider.get_backend("ibm_nairobi")

optimization_level = 0
shots = 1000

folder_path = f"data/{experiment_id}"

if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists(folder_path):
    os.mkdir(folder_path)

In [3]:
def run_and_measure(circuit: QuantumCircuit, draw: bool = False) -> dict[str, int]:
    """Transpiles and runs the QuantumCircuit, and returns the resulting counts of the execution."""
    if draw:
        print(circuit.draw())
    transpiled_circuit = transpile(
        circuit, backend=backend, optimization_level=optimization_level)
    if draw:
        print(transpiled_circuit.draw())
    result = backend.run(transpiled_circuit, shots=shots).result()
    counts = result.get_counts()
    # Reverse the string since the most significant qubit is at the 0th index of the measurement string
    reversed_keys_dict = {}
    for key, value in counts.items():
        reversed_keys_dict[key[::-1]] = value
    return reversed_keys_dict

In [4]:
# Encrypt 
global_params = GlobalParameters.generate_from_lambda(1)
key = Key.generate_key(global_params)
message = random_bit_string(global_params.n)
ciphertext = encrypt(message, key, global_params)
original_circuit = ciphertext.circuit.copy() # We might modify this circuit in subsequent steps

# Write states to files
with open(f"{folder_path}/{experiment.experiment_attributes_filename}", "w") as f:
    f.write(json.dumps({
        "experiment_id": experiment_id, 
        "execution_datetime": execution_datetime.isoformat(),
        "execution_shots": shots,
        "backend_system": system_string,
        "folder_path": folder_path,
    }))

with open(f"{folder_path}/{experiment.parameters_filename}", "w") as f:
    f.write(global_params.to_json())

with open(f"{folder_path}/{experiment.key_filename}", "w") as f:
    f.write(key.to_json())

with open(f"{folder_path}/{experiment.ciphertext_filename}", "w") as f:
    f.write(ciphertext.to_json())

with open(f"{folder_path}/{experiment.message_filename}", "w") as f:
    f.write(message)

with open (f"{folder_path}/{experiment.circuit_filename}", "wb") as f:
    qpy_serialization.dump(original_circuit, f)

In [5]:
# Test 1 - honest delete
deletion_circuit_test1 = delete(ciphertext)
deletion_counts_test1 = run_and_measure(deletion_circuit_test1)
export_counts(deletion_counts_test1, csv_filename=f"{folder_path}/{experiment.test1_filename}", key_label="Deletion measurement")

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [6]:
# Test 2 - decrypt
decryption_circuit_test2 = create_decryption_circuit(key, ciphertext)
decryption_counts_test2 = run_and_measure(decryption_circuit_test2)

export_counts(decryption_counts_test2, csv_filename=f"{folder_path}/{experiment.test2_filename}", key_label="Decryption measurement")

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [7]:
# Test 3 - honest delete and then decrypt
deletion_circuit_test3 = delete(ciphertext)
ciphertext.circuit = deletion_circuit_test3 # To use the new circuit as the starting point of the decryption_measurement circuit
decryption_circuit_test3 = create_decryption_circuit(key, ciphertext)
raw_counts_test3 = run_and_measure(decryption_circuit_test3)

export_counts(raw_counts_test3, csv_filename=f"{folder_path}/{experiment.test3_filename}", key_label="Measurement")

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [8]:
# Test 4 - malicious delete and then decrypt
deletion_circuit_test4 = breidbart_measurement(ciphertext)
ciphertext.circuit = deletion_circuit_test4 # To use the new circuit as the starting point of the decryption_measurement circuit
decryption_circuit_test4 = create_decryption_circuit(key, ciphertext)
raw_counts_test4 = run_and_measure(decryption_circuit_test4)

export_counts(raw_counts_test4, csv_filename=f"{folder_path}/{experiment.test4_filename}", key_label="Measurement")

ciphertext.circuit = original_circuit.copy() # To make it easier to re-run cells

In [9]:
# Create zip file
shutil.make_archive("data_export", "zip", "data");